In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1662547440890_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName('etl')\
    .config('spark.jars.packages', '/usr/share/aws/redshift/jdbc/RedshiftJDBC.jar,/usr/share/aws/redshift/spark-redshift/lib/spark-redshift.jar,/usr/share/aws/redshift/spark-redshift/lib/spark-avro.jar,/usr/share/aws/redshift/spark-redshift/lib/minimal-json.jar')\
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
import pyspark.sql.window as w

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Connection config to Redshift
username= ''
password = ''
jdbc_url =''
port='5439'
url = f"{jdbc_url}:{port}/dev?user=" + username + "&password=" + password
s3_tempdir = 's3://patents-analytics/tempdir/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ETL for date table

### Read cleaned patent table

In [70]:
patent_cleaned_path = "s3://patents-analytics/cleaned/patent.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
patent_cleaned = spark.read.parquet(patent_cleaned_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
patent_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- year: integer (nullable = true)

### Transform the data

In [73]:
patent_cleaned_date = patent_cleaned.select("date").distinct().select("date", F.year("date").alias('year'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
rows = patent_cleaned_date.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load data to Redshift

In [75]:
patent_cleaned_date.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "dates")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ETL for Details Dimension

## Transform

In [24]:
details = patent_cleaned.select(F.col("id").alias("detail_id"), "title", "abstract")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
details.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+
|detail_id|               title|            abstract|
+---------+--------------------+--------------------+
|  6849261|CD4-independent H...|The invention rel...|
|  6936069|Muscle stripping ...|A tendon graft st...|
|  6963268|Process for the a...|This process rela...|
|  6912482|Data storage anal...|System and method...|
|  6974555|Method of fabrica...|A method of fabri...|
|  6940399|Tire air pressure...|A tire air pressu...|
|  6941795|Method for evalua...|A method for eval...|
|  6863343|Vehicle headrest ...|A vehicle headres...|
|  6877432|Delivery of emuls...|Apparatus for del...|
|  6931964|Transfer case wit...|A transfer case i...|
|  6916013|Device for exerti...|A device for exer...|
|  6925251|Optical disc appa...|In optical disc a...|
|  6921473|Electrolytic redu...|A method of remov...|
|  6883479|VCT phaser having...|A locking device ...|
|  6837641|Securable writing...|A securable writi...|
|  6878607|Thin film device 

In [26]:
details.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5718002

### Load to Redshift

In [39]:
s3_tempdir = 's3://patents-analytics/tempdir/details/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
import math

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
DEFAULT_TEXT_COL_LENGTH = 256
MAX_TEXT_COL_LENGTH = 65535

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
# To workaround issue where spark-redshift library assume the column type to be TEXT varchar(256) for all the strings
# https://gist.github.com/pallavi/f83a45308ba8387f6b227c28aa209077
def prepare_for_write(df):
    for (column, type) in df.dtypes:
    
        if type == "string":
          max_length = df.select(F.length(df[column]).alias("length")) \
            .select(F.max("length")) \
            .first()["max(length)"]
          if max_length != None and max_length > DEFAULT_TEXT_COL_LENGTH:
            col_length = int(pow(2, math.ceil(math.log(max_length)/math.log(2)))) # round up to a power of 2
            col_length = min(MAX_TEXT_COL_LENGTH, col_length)
            df = df.withColumn(column, df[column].substr(0, col_length).alias(column, metadata={ "maxlength": col_length })) # truncate column values in case they are longer than Redshift's max allowed length
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
details_to_write = prepare_for_write(details)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
details_to_write.select(F.max(F.length("abstract"))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+
|max(length(abstract))|
+---------------------+
|                14098|
+---------------------+

In [94]:
details_to_write.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "details")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ETL for WIPOClassifications Dimension

### Extract the Wipo Table and Wipo Field Table

In [41]:
wipo_path= "s3://patents-analytics/cleaned/wipo.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
wipo_field_path =  "s3://patents-analytics/cleaned/wipo_field.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
wipo_data = spark.read.parquet(wipo_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
wipo_field_data = spark.read.parquet(wipo_field_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
wipo_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: integer (nullable = true)
 |-- field_id: double (nullable = true)
 |-- sequence: integer (nullable = true)

In [100]:
wipo_field_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- sector_title: string (nullable = true)
 |-- field_title: string (nullable = true)

### Transform

In [102]:
wipo_data = wipo_data.join(wipo_field_data, wipo_data.field_id == wipo_field_data.id).select(F.col("patent_id").alias("wipo_classification_id"), F.col("sector_title").alias("sector"), F.col("field_title").alias("field"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:
wipo_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+--------------------+--------------------+
|wipo_classification_id|              sector|               field|
+----------------------+--------------------+--------------------+
|               5003481|Mechanical engine...|           Transport|
|               6156689|           Chemistry|Basic materials c...|
|               6071282|         Instruments|  Medical technology|
|              10763770|Electrical engine...|Electrical machin...|
|               8779043|           Chemistry|Macromolecular ch...|
|              10103124|Electrical engine...|      Semiconductors|
|               7042419|Electrical engine...|  Telecommunications|
|               8847338|Electrical engine...|      Semiconductors|
|              11090079|         Instruments|  Medical technology|
|               6433583|Electrical engine...| Computer technology|
|               8831205|Electrical engine...|  Telecommunications|
|               5749401|Mechanical engine...|            Handl

In [105]:
wipo_data.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "wipo_classifications")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
wipo_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7221215

## ETL for Owners Dimension

### Read patent cleaned

In [5]:
patent_cleaned_path = "s3://patents-analytics/cleaned/patent.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
patent_cleaned = spark.read.parquet(patent_cleaned_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
patent_cleaned.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- abstract: string (nullable = true)
 |-- title: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- num_claims: integer (nullable = true)
 |-- year: integer (nullable = true)

In [8]:
patent_cleaned.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5718002

### Read patent assignee

In [9]:
patent_assignee_path = "s3://patents-analytics/cleaned/patent_assignee.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
patent_assignee = spark.read.parquet(patent_assignee_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
patent_assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: string (nullable = true)
 |-- assignee_id: string (nullable = true)
 |-- location_id: string (nullable = true)

### Read assignee

In [12]:
assignee_path = "s3://patents-analytics/cleaned/assignee.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
assignee = spark.read.parquet(assignee_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- new_id: string (nullable = true)

### Read patent inventor

In [15]:
patent_inventor_path = "s3://patents-analytics/cleaned/patent_inventor.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
patent_inventor = spark.read.parquet(patent_inventor_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
patent_inventor.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- patent_id: string (nullable = true)
 |-- inventor_id: string (nullable = true)
 |-- location_id: string (nullable = true)

### Read inventor

In [18]:
inventor_path = "s3://patents-analytics/cleaned/inventor.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
inventor = spark.read.parquet(inventor_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
inventor.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- new_id: string (nullable = true)

### Read location

In [21]:
location_path = "s3://patents-analytics/cleaned/location.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
location = spark.read.parquet(location_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
location.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- new_id: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Continent: string (nullable = true)

In [24]:
location.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------+---------+
|                  id|              new_id|       Country|Continent|
+--------------------+--------------------+--------------+---------+
|ef226adc-cb90-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|55144e66-cb90-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|d8235ebd-cb90-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|2874158a-cb8f-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|540ca7ca-cb90-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|4163c23a-09bc-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|a8faf4ef-cb8f-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|c187b518-09bd-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|cf18721f-cb8f-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|774addd0-cb90-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|6e70bb1d-cb8e-11e...|8d23a6e37e0a6431a...|United Kingdom|       EU|
|9aa079ee-cb90-11e...|8d23a6e37e0a

In [25]:
location.filter(F.col('new_id').isNull()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+-------+---------+
| id|new_id|Country|Continent|
+---+------+-------+---------+
+---+------+-------+---------+

### Join patent and patent assignee

In [26]:
patent_owner = patent_cleaned.join(patent_assignee, patent_cleaned.id == patent_assignee.patent_id, "left").select("id", "assignee_id", "location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
patent_with_assignee = patent_owner.filter(F.col("assignee_id").isNotNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
patent_without_assignee = patent_owner.filter(F.col("assignee_id").isNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
patent_with_assignee.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+
|      id|         assignee_id|         location_id|
+--------+--------------------+--------------------+
|10000001|2f57ff76-4e82-475...|e8ec5703-cb8f-11e...|
|10000040|64187300-4f51-4a5...|f643dd2b-09bb-11e...|
|10000043|e82e8525-bd3c-4ad...|ff5c6634-cb8f-11e...|
|10000099|dd2d5dd4-0b6d-400...|f0c35034-cb90-11e...|
|10000136|9d5a4c76-7be9-4b3...|ff7e35db-cb90-11e...|
|10000144|9634df64-1c8c-435...|f5538e3f-cb8e-11e...|
|10000212|dd6c5836-4509-41c...|ff7e35db-cb90-11e...|
|10000231|9a1187e7-c389-442...|fa457e8d-cb90-11e...|
|10000235|dce088dd-709b-418...|ff4052ad-cb90-11e...|
|10000341|a311f674-a619-419...|f99338de-cb8e-11e...|
|10000354|87a2bb11-4700-479...|ff405b60-cb8e-11e...|
|10000427|94a7a434-bbca-484...|fadd6575-09bd-11e...|
|10000463|3675d2cd-33f3-4e2...|f16c2e15-cb8e-11e...|
|10000477|d4dee247-3ec8-430...|f22e1e81-cb8e-11e...|
|10000479|9ae4997d-1a45-488...|f9fd6a5c-cb8e-11e...|
|10000490|c215870d-1c87-419...|f9fd6a5c-cb8e-1

In [30]:
patent_with_assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5234543

In [31]:
patent_without_assignee.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------+-----------+
|      id|assignee_id|location_id|
+--------+-----------+-----------+
|10000283|       null|       null|
|10000304|       null|       null|
|10000887|       null|       null|
|10001015|       null|       null|
|10001287|       null|       null|
|10001289|       null|       null|
|10002545|       null|       null|
|10002552|       null|       null|
|10002595|       null|       null|
|10003886|       null|       null|
|10003940|       null|       null|
|10004186|       null|       null|
|10004349|       null|       null|
|10004944|       null|       null|
|10005163|       null|       null|
|10005329|       null|       null|
|10005804|       null|       null|
|10006197|       null|       null|
|10006385|       null|       null|
|10006743|       null|       null|
+--------+-----------+-----------+
only showing top 20 rows

In [32]:
patent_without_assignee.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

483459

### Join patent with assignee with assignee

In [33]:
patent_with_assignee = patent_with_assignee.join(assignee, patent_with_assignee.assignee_id==assignee.id, "left").drop(assignee.id).select("id", F.col("new_id").alias("owner_id"), "name", "type", "location_id").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
patent_with_assignee.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+----------+--------------------+
|      id|            owner_id|                name|      type|         location_id|
+--------+--------------------+--------------------+----------+--------------------+
| D877542|deb57160757b2f0bb...|Attendance on Dem...|   company|c41fcefa-cb8e-11e...|
| D874679|deb57160757b2f0bb...|Attendance on Dem...|   company|c41fcefa-cb8e-11e...|
| 7179436|0668165141eddab5a...|     Gregory L. Noss|individual|fac6a96e-cb8f-11e...|
| 7287558|924220f93869caaa8...|Arizona Public Se...|   company|ebbac1b8-cb8e-11e...|
| 7275569|924220f93869caaa8...|Arizona Public Se...|   company|ebbac1b8-cb8e-11e...|
| 7647924|924220f93869caaa8...|Arizona Public Se...|   company|ebbac1b8-cb8e-11e...|
| 8043847|924220f93869caaa8...|Arizona Public Se...|   company|ebbac1b8-cb8e-11e...|
| 6352645|924220f93869caaa8...|Arizona Public Se...|   company|ebbac1b8-cb8e-11e...|
| 6792200|924220f93869caaa8...|Arizona Public Se...|   company|eb

### Join patent with assignee with location

In [35]:
patent_with_assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- owner_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- location_id: string (nullable = true)

In [36]:
patent_with_assignee_location = patent_with_assignee.join(location, patent_with_assignee.location_id == location.id, "left").drop(location.id).select("id", "owner_id", "name", "type", F.col("new_id").alias("location_id"), F.col("Country").alias("country"), F.col("Continent").alias("continent"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
patent_with_assignee_location.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+----------+--------------------+-------------+---------+
|      id|            owner_id|                name|      type|         location_id|      country|continent|
+--------+--------------------+--------------------+----------+--------------------+-------------+---------+
| D877542|deb57160757b2f0bb...|Attendance on Dem...|   company|49dca65f362fee401...|United States|       NA|
| D874679|deb57160757b2f0bb...|Attendance on Dem...|   company|49dca65f362fee401...|United States|       NA|
| 7179436|0668165141eddab5a...|     Gregory L. Noss|individual|49dca65f362fee401...|United States|       NA|
| 7287558|924220f93869caaa8...|Arizona Public Se...|   company|49dca65f362fee401...|United States|       NA|
| 7275569|924220f93869caaa8...|Arizona Public Se...|   company|49dca65f362fee401...|United States|       NA|
| 7647924|924220f93869caaa8...|Arizona Public Se...|   company|49dca65f362fee401...|United States|       NA|
| 8043847|924220f93

In [38]:
patent_with_assignee_location.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5234543

### Join patent without assignee with patent inventor

In [39]:
patent_without_assignee.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- assignee_id: string (nullable = true)
 |-- location_id: string (nullable = true)

In [40]:
patent_without_assignee_join_inventor = patent_without_assignee.join(patent_inventor, patent_without_assignee.id == patent_inventor.patent_id, "left").drop(patent_without_assignee.location_id).select("id", "inventor_id", "location_id").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
patent_without_assignee_join_inventor.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+
|      id|         inventor_id|         location_id|
+--------+--------------------+--------------------+
|10000304|  fl:ar_ln:moncayo-1|4fa58d4b-cb90-11e...|
|10001331|    fl:wi_ln:couie-1|74826f5b-cb8e-11e...|
|10011501|  fl:ma_ln:clifton-2|795acf3b-cb8e-11e...|
|10016361|   fl:fr_ln:farron-1|ffe7bef3-cb8e-11e...|
|10017297|     fl:ro_ln:cai-14|ed7cdd0a-cb90-11e...|
|10018738|fl:em_ln:kamienie...|462b1099-cb8e-11e...|
|10029138|fl:do_ln:boatwrig...|cf8cf64a-cb8e-11e...|
|10029165|  fl:ty_ln:kloster-1|3cddd01f-cb8e-11e...|
|10036347|   fl:ra_ln:sherry-2|5f48fd79-cbae-11e...|
|10036564|    fl:te_ln:daley-1|f751ef48-cb8e-11e...|
|10045609|he7mnuftnge5ftsih...|c76d1a3e-cb8e-11e...|
|10051937|   fl:ch_ln:wenzel-7|d08fc279-09bd-11e...|
|10051969| fl:na_ln:galloway-1|efa16d0b-cb8f-11e...|
|10052445|fl:ga_ln:campbell...|f3337288-cb90-11e...|
|10053126| fl:to_ln:buttimer-1|ed842009-cb90-11e...|
|10058721|    fl:je_ln:dykes-1|f498a170-cb8e-1

### Join patent without assignee with inventor

In [42]:
patent_without_assignee_join_inventor2 = patent_without_assignee_join_inventor.join(inventor, patent_without_assignee_join_inventor.inventor_id == inventor.id, "left").drop(inventor.id).select("id", "name", "type", F.col("new_id").alias("owner_id"), "location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
patent_without_assignee_join_inventor2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----------+--------------------+--------------------+
|      id|                name|      type|            owner_id|         location_id|
+--------+--------------------+----------+--------------------+--------------------+
| 8091846|      John D Britner|individual|0cb6d66f951759dca...|3d200c9e-cb8e-11e...|
| D777352|         He Dengying|individual|86a230ec9fe3141ce...|fd0172ae-09be-11e...|
| 7140501|     Clayton Shumate|individual|9653b2d1f2ebfb096...|25f4c6d0-cb90-11e...|
| D478967|    Solomon G. Harry|individual|46cb612ab58360dc9...|e295c332-cb8e-11e...|
| D489642|      Sharon Brumlow|individual|65efb4dfb9da5467b...|a461430b-cb8e-11e...|
| 6723502|      Antonio Samson|individual|158d87092f9fe6b30...|3cb8f3df-cb8e-11e...|
| 8011284|     Benzoni Rinaldo|individual|fd6d9e73265ea562b...|dfc3f45e-cb8e-11e...|
| D419313|  Abraham I. Gelbart|individual|a84150039831d536f...|eeb89da2-09be-11e...|
|11140968|Robert M. Lipani,...|individual|54841b2d0d2729be6...|cd

### Join patent without assignee with location

In [44]:
patent_without_assignee_join_inventor2_loc = patent_without_assignee_join_inventor2.join(location,patent_without_assignee_join_inventor2.location_id == location.id, "left" ).drop(location.id).select("id", "owner_id", "name", "type", F.col("new_id").alias("location_id"), F.col("Country").alias("Country"), F.col("Continent").alias("continent"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
patent_without_assignee_join_inventor2_loc.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+----------+--------------------+-------------+---------+
|      id|            owner_id|                name|      type|         location_id|      Country|continent|
+--------+--------------------+--------------------+----------+--------------------+-------------+---------+
| 8091846|0cb6d66f951759dca...|      John D Britner|individual|49dca65f362fee401...|United States|       NA|
| D777352|86a230ec9fe3141ce...|         He Dengying|individual|10436829032f361a3...|        China|       AS|
| 7140501|9653b2d1f2ebfb096...|     Clayton Shumate|individual|49dca65f362fee401...|United States|       NA|
| D478967|46cb612ab58360dc9...|    Solomon G. Harry|individual|49dca65f362fee401...|United States|       NA|
| D489642|65efb4dfb9da5467b...|      Sharon Brumlow|individual|49dca65f362fee401...|United States|       NA|
| 6723502|158d87092f9fe6b30...|      Antonio Samson|individual|49dca65f362fee401...|United States|       NA|
| 8011284|fd6d9e732

In [46]:
patent_without_assignee_join_inventor2_loc.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

483459

### Combine patent with assignee and patent without assignee

In [47]:
patent_combined = patent_with_assignee_location.union(patent_without_assignee_join_inventor2_loc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
patent_combined.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+----------+--------------------+-------------+---------+
|      id|            owner_id|                name|      type|         location_id|      country|continent|
+--------+--------------------+--------------------+----------+--------------------+-------------+---------+
| D877542|deb57160757b2f0bb...|Attendance on Dem...|   company|49dca65f362fee401...|United States|       NA|
| D874679|deb57160757b2f0bb...|Attendance on Dem...|   company|49dca65f362fee401...|United States|       NA|
| 7179436|0668165141eddab5a...|     Gregory L. Noss|individual|49dca65f362fee401...|United States|       NA|
| 7287558|924220f93869caaa8...|Arizona Public Se...|   company|49dca65f362fee401...|United States|       NA|
| 7275569|924220f93869caaa8...|Arizona Public Se...|   company|49dca65f362fee401...|United States|       NA|
| 7647924|924220f93869caaa8...|Arizona Public Se...|   company|49dca65f362fee401...|United States|       NA|
| 8043847|924220f93

In [49]:
patent_combined.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5718002

### Load Owner Table

In [50]:
owner = patent_combined.select("owner_id", "type", "name").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
owner2 = owner.filter(F.col("owner_id").isNotNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
owner2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|            owner_id|      type|                name|
+--------------------+----------+--------------------+
|a21665f1cfe64e179...|   company|Lightspeed Semico...|
|baa277a81335916e9...|   company|New Affinity Conc...|
|a6ffea4e3ca0230d6...|   company|John J. Pemboke L...|
|feb17eb9101af0f63...|   company|    RNCommerce, Inc.|
|97db85c4c92aefcce...|   company|Breeze-Torce Prod...|
|32f8892789b9e6e62...|   company|FUISZ HNB TECHNOL...|
|cff099f5942dcbc82...|   company|VistaLab Technolo...|
|6e00f5e295e070f55...|   company|    PHARMA MAR, S.A.|
|1ac468b579ef69daf...|   company|          LuxAI S.A.|
|d2455ee81c36c31a2...|   company|       PIP Co., Ltd.|
|b701b1f5d8cc8fbd0...|   company|MILBRANK MANUFACT...|
|24498123f3def598f...|   company|   H3 Platform, Inc.|
|3b2975d432fc39cde...|   company|Handle Financial,...|
|717bb481efe0652fb...|   company|ERDA MASTER IPCO ...|
|79ec08fecd707a60c...|   company|Lih Rurng Instrum...|
|19869e9a3

In [53]:
owner2.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "owners")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load Location Data

In [54]:
location = patent_combined.select("location_id", "country", "continent").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
location.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---------+
|         location_id|             country|continent|
+--------------------+--------------------+---------+
|7f69649ad70496d44...|              Poland|       EU|
|55ccc5014f2a6d4c4...|               Qatar|       AS|
|94b2859b913c9e2cf...|             Bahrain|       AS|
|49dca65f362fee401...|       United States|       NA|
|                null|                null|     null|
|c408d26c1ec1301a8...|         El Salvador|       NA|
|a2ea3384e02f917ad...|            Slovakia|       EU|
|6ff080c07894909d1...|       Liechtenstein|       EU|
|2275583196d791405...|         Switzerland|       EU|
|71dc81f13c2d66fa5...|              Belize|       NA|
|0e89123f46fcba83a...|Saint Kitts and N...|       NA|
|c3668f581289444ea...|              Panama|       NA|
|c65742ceb289d67f6...|    Papua New Guinea|       OC|
|7e222a78e771f1b9b...|               Gabon|       AF|
|c387e2d50fff5b38c...|    Falkland Islands|       SA|
|abe06aae4495f3cf8...|      

In [56]:
location2 = location.filter(F.col("location_id").isNotNull())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
location2.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "locations")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load the Patent Fact table

In [56]:
patent_with_owner_location = patent_combined.select("id", "owner_id", "location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
patent_with_all_fields = patent_with_owner_location.join(patent_cleaned, patent_with_owner_location.id == patent_cleaned.id).drop(patent_cleaned.id).select("id",F.col("date").alias("granted_date"), F.col("id").alias("detail_id"), "owner_id", "location_id",F.col("id").alias("wipo_classification_id"), "num_claims"  )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
patent_with_all_fields.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+---------+--------------------+--------------------+----------------------+----------+
|      id|       granted_date|detail_id|            owner_id|         location_id|wipo_classification_id|num_claims|
+--------+-------------------+---------+--------------------+--------------------+----------------------+----------+
|10000001|2018-06-19 00:00:00| 10000001|1811ccc2f6d60eac3...|fa34f97bf4bcb580f...|              10000001|        12|
|10000040|2018-06-19 00:00:00| 10000040|85f93d76014d5588f...|50326770c34ca13ba...|              10000040|        17|
|10000043|2018-06-19 00:00:00| 10000043|d5e078a70f7ee0732...|501c26b2571a1f7fd...|              10000043|        19|
|10000099|2018-06-19 00:00:00| 10000099|9ec77b49813b21610...|6814ef46f686990cf...|              10000099|        14|
|10000136|2018-06-19 00:00:00| 10000136|25a9b70e4a769fb37...|fa34f97bf4bcb580f...|              10000136|        21|
|10000144|2018-06-19 00:00:00| 10000144|8afd7a5207031e796...|8a9

In [60]:
patent_with_all_fields.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5718002

In [61]:
patent_with_all_fields.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "patents")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ETL for Patent Keywords

### Load patent keyword

In [63]:
patent_keyword_path = "s3://patents-analytics/cleaned/patent_keyword.parquet/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
patent_keyword = spark.read.parquet(patent_keyword_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
patent_keyword.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- score: string (nullable = true)
 |-- row: integer (nullable = true)

### Transform

In [67]:
patent_keyword2 = patent_keyword.select(F.sha2(F.concat(F.col("id"), F.col("keyword")),256).alias("patent_keyword_id"), F.col("id").alias("patent_id"), "keyword")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
patent_keyword2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-------------------+
|   patent_keyword_id|patent_id|            keyword|
+--------------------+---------+-------------------+
|c85df9e841b7b26ac...| 10000057|             nozzle|
|ac95cb1259eacb4c2...| 10000057|         nozzle set|
|6450c2ddafdc06116...| 10000057|       first nozzle|
|55a82174ee8579de4...| 10000057|              print|
|27ee2715256ab679a...| 10000057|                set|
|45ff2df52c5d2e117...| 10000057|              first|
|4774307f4c54803bf...| 10000057|         print head|
|2293f3a85de28cb21...| 10000057|           printing|
|d301a8c72b5013dad...| 10000057|        information|
|bd2be7d682955fea7...| 10000057|               head|
|5e17f86f567f7e9db...| 10000517|          complexes|
|2bbf17582b02a0a42...| 10000638|             weight|
|1a7f91409c7f504cd...| 10000638|            asphalt|
|b12523faafadddcc4...| 10000638|asphalt composition|
|d772b6bc373e37fbd...| 10000638|  synthetic rubbers|
|dc2249b4b9fcee96f...| 10000638|   modified as

### Load

In [69]:
patent_keyword2.write\
    .format("io.github.spark_redshift_community.spark.redshift")\
    .option("url", url)\
    .option("tempdir", s3_tempdir)\
    .option("dbtable", "patent_keywords")\
    .mode("overwrite")\
    .option("aws_iam_role", "arn:aws:iam::735201909841:role/redshift_read_s3")\
    .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…